## 👉08-03. 토치텍스트 튜토리얼 - 한국어

## 1. 형태소 분석기 Mecab 설치

In [1]:
# 🔸Ubuntu cmd 창에 입력하기🔸
# sudo yum install curl git
# bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

## 2. 훈련 데이터와 테스트 데이터로 다운로드 하기

In [2]:
import urllib.request
import pandas as pd

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7faee778ca30>)

In [4]:
train_df = pd.read_table("ratings_train.txt")
test_df = pd.read_table("ratings_test.txt")

In [5]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [7]:
print("훈련 데이터 샘플의 개수 : {}".format(len(train_df)))
print("테스트 데이터 샘플의 개수 : {}".format(len(test_df)))

훈련 데이터 샘플의 개수 : 150000
테스트 데이터 샘플의 개수 : 50000


## 3. 필드 정의하기 (torchtext.legacy.data)

In [8]:
from torchtext.legacy import data 
from konlpy.tag import Mecab

In [9]:
tokenizer = Mecab()

In [10]:
ID = data.Field(sequential=False,
               use_vocab=False)

TEXT = data.Field(sequential=True,
                 use_vocab=True,
                 tokenize=tokenizer.morphs,
                 lower=True,
                 batch_first=True,
                 fix_length=20)

LABEL = data.Field(sequential=False,
                  use_vocab=False,
                  is_target=True)

## 4. 데이터셋 만들기

In [11]:
from torchtext.legacy.data import TabularDataset

In [12]:
train_data, test_data = TabularDataset.splits(path='.',
                                             train="ratings_train.txt",
                                             test="ratings_test.txt",
                                             format="tsv",
                                             fields=[("id", ID), ("text", TEXT), ("label", LABEL)],
                                             skip_header=True)

In [13]:
print(vars(train_data[0]))

{'id': '9976970', 'text': ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], 'label': '0'}


## 5. 단어 집합(Vocabulary) 만들기

In [14]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [15]:
print("단어 집합의 크기 : {}".format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


In [16]:
print(TEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fae769994c0>>, {'<unk>': 0, '<pad>': 1, '.': 2, '이': 3, '는': 4, '영화': 5, '다': 6, '고': 7, '하': 8, '도': 9, '의': 10, '가': 11, '은': 12, '에': 13, '을': 14, '한': 15, '보': 16, '..': 17, '게': 18, ',': 19, '들': 20, '!': 21, '지': 22, '를': 23, '있': 24, '없': 25, '?': 26, '좋': 27, '나': 28, '었': 29, '만': 30, '는데': 31, '너무': 32, '봤': 33, '적': 34, '안': 35, '로': 36, '정말': 37, '음': 38, '으로': 39, '것': 40, '아': 41, '네요': 42, '재밌': 43, '어': 44, '점': 45, '같': 46, '지만': 47, '진짜': 48, '했': 49, '기': 50, '에서': 51, '네': 52, '않': 53, '거': 54, '았': 55, '수': 56, '되': 57, '면': 58, 'ㅋㅋ': 59, '과': 60, '말': 61, '인': 62, '연기': 63, '잘': 64, '최고': 65, '주': 66, '내': 67, '~': 68, '평점': 69, '이런': 70, '어요': 71, '던': 72, '와': 73, '할': 74, '왜': 75, '1': 76, '겠': 77, '해': 78, '스토리': 79, '습니다': 80, 'ㅋㅋㅋ': 81, '...': 82, '드라마': 83, '아니': 84, '생각': 85, '더': 86, '그': 87, '싶': 88, '사람': 89, '듯': 90, '감동': 91, '때': 92, '함': 93, '배우': 94, '본': 95, 

## 6. 토치텍스트의 데이터로더 만들기

In [17]:
from torchtext.legacy.data import Iterator

In [18]:
batch_size = 5

In [19]:
train_loader = Iterator(dataset=train_data,
                       batch_size=batch_size)
test_loader = Iterator(dataset=test_data,
                      batch_size=batch_size)

In [20]:
print("훈련 데이터의 미니 배치 수 : {}".format(len(train_loader)))
print("테스트 데이터의 미니 배치 수 : {}".format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 30000
테스트 데이터의 미니 배치 수 : 10000


In [21]:
batch = next(iter(train_loader))
print(batch.text)

tensor([[ 138,  156,   51,  792,  502,  727,    4,  193, 1332,    0,    2,   17,
            1,    1,    1,    1,    1,    1,    1,    1],
        [1116,    5,    6,   91,   12,  147,    8,   22,   61,  105,  798,   91,
           14,   66,  413,   49,    6,    2,  168,    3],
        [ 157,  827,   10, 1693,   60,  605,   15,  118,  172,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [ 145,  109,    3,  108,    8,   22,   53,   55,    6,    2,   17,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [3973,   21,   21, 4725,   20, 3522,   61,    7,   21,   76,    0,    4,
         1662,   15, 3973,   10,    0, 1359,   21,  203]])
